In [1]:
import requests
import json
import pandas as pd
import time

In [2]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, indent=4)
    print(text)

## Offene Gesetze

Let's down load all the metadata for all the laws! 

Saving it into CSV and dataframe for now

Later on need to write a function to grab updates from the api but this corpus is enough for our prototyping needs

In [118]:
# of_response = [requests.get("https://api.offenegesetze.de/v1/veroeffentlichung/?year="+str(year)) for year in range(1949,2022)]

In [ ]:
x = 0
og_response = []
og_response.append(requests.get("https://api.offenegesetze.de/v1/veroeffentlichung/"))
next_page = og_response[-1].json()["next"]
while next_page != "None":
    og_response.append(requests.get(next_page))
    next_page = og_response[-1].json()["next"]
    x += 1
    print(
        next_page, x
    )  # added for testing today because there's an API malefunction. remove when things get back to normal
    if x % 15 == 0:
        time.sleep(1)

In [135]:
# OffeneGesetze_df_list = [pd.DataFrame.from_dict(of_response[year].json()["results"]) for year in range(0,73)]
OffeneGesetze_df_list = [
    pd.DataFrame.from_dict(og_response[page].json()["results"])
    for page in range(0, x + 1)
]

In [136]:
OffeneGesetze_df = pd.concat(OffeneGesetze_df_list, ignore_index=True)

In [140]:
OffeneGesetze_df.to_csv(
    index=False, path_or_buf="../data/source_laws/OffeneGesetzeAPI.csv"
)

In [ ]:
# list of dictionaries, can be converted ot pandas dataframe with from_dict - Done above yay!
jprint(of_response[72].json()["results"])

In [ ]:
# just a quick check to see if I have reasonable amounts of data for each year
[len(OffeneGesetze_df[OffeneGesetze_df["year"] == yr]) for yr in range(1949, 2022)]

In [141]:
OffeneGesetze_df.head()

,id,kind,year,number,date,url,api_url,document_url,order,title,law_date,page,pdf_page,num_pages
0,bgbl1-2021-74-1,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,1,Dreiundsechzigste Verordnung zur Durchführung ...,2021-10-07T00:00:00Z,4682.0,2,1
1,bgbl1-2021-74-2,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,2,Zweite Verordnung zur Änderung der Weinverordn...,2021-10-11T00:00:00Z,4683.0,3,5
2,bgbl1-2021-74-3,bgbl1,2021,74,2021-10-19T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,3,Erste Verordnung zur Änderung der Bußgeldkatal...,2021-10-13T00:00:00Z,4688.0,8,16
3,bgbl1-2021-73-1,bgbl1,2021,73,2021-10-18T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,1,Gesetz zur Verbesserung der Transparenzregeln ...,2021-10-08T00:00:00Z,4650.0,2,5
4,bgbl1-2021-73-2,bgbl1,2021,73,2021-10-18T00:00:00Z,https://offenegesetze.de/veroeffentlichung/bgb...,https://api.offenegesetze.de/v1/veroeffentlich...,https://media.offenegesetze.de/bgbl1/2021/bgbl...,2,Verordnung zur Stärkung der Organisationen und...,2021-10-11T00:00:00Z,4655.0,7,12


## DIP

Let's download the proposals

The API key until end of May 2022 is: `N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464`

Auf Anfrage können Sie auch einen eigenen API Key erhalten, der zeitlich unbefristet ist.

Bitte stellen Sie eine diesbezügliche Anfrage per E-Mail an: infoline.id3@bundestag.de

### technical info about the API

Each call is limited to 50 items in the reply. A reply will also include an item called a 'cursor' which help to keep track of queries that require multiple calls (answers consist of more than 50 items). You keep querying the API with the same parameters, adding the previous cursor you got, until the new cursor matches the one you have, at which point you know there aren't any more items and your query is complete. 

I've implemented this logic with a while loop for now. It's a bit slow but in the future we will only need an update call so I'm not too worried about optimisaiton for now.

In [3]:
cursor_check = 42
x = 0
dip_response = []
dip_response.append(
    requests.get(
        "https://search.dip.bundestag.de/api/v1/drucksache?f.zuordnung=BT&f.datum.start=2015-01-&apikey=N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464"
    )
)
cursor_new = dip_response[-1].json()["cursor"].replace("+", "%2B").replace("/", "%2F")
print(cursor_new)
while cursor_new != cursor_check:
    cursor_check = cursor_new
    dip_response.append(
        requests.get(
            "https://search.dip.bundestag.de/api/v1/drucksache?f.zuordnung=BT&f.datum.start=2015-01&apikey=N64VhW8.yChkBUIJeosGojQ7CSR2xwLf3Qy7Apw464&cursor="
            + cursor_new
        )
    )
    cursor_new = (
        dip_response[-1].json()["cursor"].replace("+", "%2B").replace("/", "%2F")
    )
    x += 1
    print(
        cursor_new, x
    )  # added for testing today because there's an API malefunction. remove when things get back to normal
    if x % 15 == 0:
        time.sleep(1)

AoJwwOKCk%2F0CMURydWNrc2FjaGUtMjU4MzIz
AoJwwNu8u%2FwCMURydWNrc2FjaGUtMjU4MTcy 1
AoJwgMHGivwCMURydWNrc2FjaGUtMjU3OTU1 2
AoJwwJmw8%2FsCMURydWNrc2FjaGUtMjU3ODE1 3
AoJwwNT24%2FsCMURydWNrc2FjaGUtMjU3NTc1 4
AoJwgPKZ3PsCMURydWNrc2FjaGUtMjU3NDc2 5
AoJwwOGWyvsCMURydWNrc2FjaGUtMjU3MzA3 6
AoJwwJzduvsCMURydWNrc2FjaGUtMjU3MjI1 7
AoJwgMegmfsCMURydWNrc2FjaGUtMjU3MTI1 8
AoJwwLadh%2FsCMURydWNrc2FjaGUtMjU3MDg3 9
AoJwwNrQ8voCMURydWNrc2FjaGUtMjU2ODg1 10
AoJwwJWX4%2FoCMURydWNrc2FjaGUtMjU2Nzgy 11
AoJwgJyn1voCMURydWNrc2FjaGUtMjU2NjU1 12
AoJwgO6AzPoCMURydWNrc2FjaGUtMjU2NTUz 13
AoJwgKnHvPoCMURydWNrc2FjaGUtMjU2NDMx 14
AoJwgOSNrfoCMURydWNrc2FjaGUtMjU2MzI0 15
AoJwgJ%2FUnfoCMURydWNrc2FjaGUtMjU2MDU1 16
AoJwwNOKm%2FoCMURydWNrc2FjaGUtMjU1OTM3 17
AoJwgIjBmPoCMURydWNrc2FjaGUtMjU1ODQ2 18
AoJwwLz3lfoCMURydWNrc2FjaGUtMjU1NzI4 19
AoJwwI7Ri%2FoCMURydWNrc2FjaGUtMjU1NTkw 20
AoJwwMmX%2FPkCMURydWNrc2FjaGUtMjU1NDQ0 21
AoJwwLKE9%2FkCMURydWNrc2FjaGUtMjU1MzIx 22
AoJwwLKE9%2FkCMURydWNrc2FjaGUtMjU1MTk4 23
AoJwgOe69PkCMURydWNrc2FjaGUt

In [4]:
# print(x, len(dip_response[-2].json()['documents']))
# >45, 1646

## last call is empty, it returns the same cursor and no documents
## one to last call returns 45 documents
## all calls before that return 100 documents per call
## so..
# 1645 * 100 +45

print("we have 164,545 document titles and linkes")

we have 164,545 document titles and linkes


In [5]:
dip_response[3].json()["documents"]

[{'id': '257940',
  'drucksachetyp': 'Kleine Anfrage',
  'dokumentart': 'Drucksache',
  'autoren_anzahl': 19,
  'typ': 'Dokument',
  'vorgangsbezug_anzahl': 1,
  'dokumentnummer': '19/32563',
  'wahlperiode': 19,
  'herausgeber': 'BT',
  'datum': '2021-09-24',
  'titel': 'Fragen zum Bau neuer Lager auf den griechischen Inseln',
  'autoren_anzeige': [{'id': '16',
    'titel': 'Ulla Jelpke, MdB, DIE LINKE',
    'autor_titel': 'Ulla Jelpke'},
   {'id': '1665',
    'titel': 'Dr. André Hahn, MdB, DIE LINKE',
    'autor_titel': 'André Hahn'}],
  'fundstelle': {'pdf_url': 'https://dserver.bundestag.de/btd/19/325/1932563.pdf',
   'id': '257940',
   'dokumentnummer': '19/32563',
   'datum': '2021-09-24',
   'verteildatum': '2021-09-28',
   'dokumentart': 'Drucksache',
   'drucksachetyp': 'Kleine Anfrage',
   'herausgeber': 'BT',
   'urheber': ['Fraktion DIE LINKE']},
  'urheber': [{'einbringer': False,
    'bezeichnung': 'LINKE',
    'titel': 'Fraktion DIE LINKE'}],
  'vorgangsbezug': [{'id': '

In [6]:
dip_response_df_list = [
    pd.DataFrame.from_dict(dip_response[cursor].json()["documents"])
    for cursor in range(0, x + 1)
]

In [7]:
len(dip_response)

1649

In [8]:
dip_response_df_list = [
    pd.json_normalize(dip_response[cursor].json()["documents"])
    for cursor in range(0, x + 1)
]

In [9]:
dip_response_df = pd.concat(dip_response_df_list, ignore_index=True)

In [10]:
dip_response_df.drucksachetypd(10)

,id,drucksachetyp,dokumentart,autoren_anzahl,typ,vorgangsbezug_anzahl,dokumentnummer,wahlperiode,herausgeber,datum,...,fundstelle.dokumentart,fundstelle.drucksachetyp,fundstelle.herausgeber,fundstelle.urheber,ressort,vorgangsbezug,autoren_anzeige,fundstelle.verteildatum,anlagen,fundstelle.anlagen
0,258476,Kleine Anfrage,Drucksache,1,Dokument,0,20/168,20,BT,2021-12-01,...,Drucksache,Kleine Anfrage,BT,[Fraktion der AfD],NaN,NaN,NaN,NaN,NaN,NaN
1,258475,Antwort,Drucksache,0,Dokument,1,20/167,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282816', 'titel': 'Nicht vollstreckte...",NaN,NaN,NaN,NaN
2,258474,Antwort,Drucksache,0,Dokument,1,20/166,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282823', 'titel': 'IS-Anhänger und de...",NaN,NaN,NaN,NaN
3,258473,Antwort,Drucksache,0,Dokument,1,20/165,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282829', 'titel': 'Flüchtlingsströme ...",NaN,NaN,NaN,NaN
4,258472,Antwort,Drucksache,0,Dokument,1,20/164,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282821', 'titel': 'Verjährungsbedingt...",NaN,NaN,NaN,NaN
5,258471,Antwort,Drucksache,0,Dokument,1,20/163,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282856', 'titel': 'Bundespolizeieinsä...",NaN,NaN,NaN,NaN
6,258470,Antwort,Drucksache,0,Dokument,1,20/162,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282852', 'titel': 'Erfassung antiziga...",NaN,NaN,NaN,NaN
7,258469,Antwort,Drucksache,0,Dokument,1,20/161,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Presse- und...","[{'id': '282822', 'titel': 'Meinungsumfragen d...",NaN,NaN,NaN,NaN
8,258468,Antwort,Drucksache,0,Dokument,1,20/159,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282824', 'titel': 'Bestandteile und V...",NaN,NaN,NaN,NaN
9,258467,Antwort,Drucksache,0,Dokument,1,20/158,20,BT,2021-12-01,...,Drucksache,Antwort,BT,[Bundesregierung],"[{'federfuehrend': True, 'titel': 'Bundesminis...","[{'id': '282830', 'titel': 'Auszahlung von För...",NaN,NaN,NaN,NaN


In [11]:
dip_response_df.to_csv(
    index=False, path_or_buf="../data/source_laws/DIP_response_API.csv"
)

## Filter for Änderungsgesetze

In [26]:
# get only recent law proposals
relevant_df = dip_response_df[
    (dip_response_df.drucksachetyp == "Gesetzentwurf")  # pick only proposals
    & (dip_response_df.datum.str.startswith("2021") | dip_response_df.datum.str.startswith("202"))        # pick only from 2021 or 2020
]

In [28]:
relevant_df["change_law"] = relevant_df.titel.map(lambda row: "Änderung" in row)

/tmp/ipykernel_2454116/1382230463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["change_law"] = relevant_df.titel.map(lambda row: "Änderung" in row)


In [36]:
change_law_df = relevant_df[relevant_df.change_law].reset_index()[["titel", "fundstelle.pdf_url"]]
change_law_df.head()

,titel,fundstelle.pdf_url
0,Entwurf eines Gesetzes zum Vorschlag für eine ...,https://dserver.bundestag.de/btd/20/001/200014...
1,Entwurf eines Gesetzes zur Änderung des Infekt...,https://dserver.bundestag.de/btd/20/000/200001...
2,Entwurf eines Gesetzes zur Errichtung eines So...,https://dserver.bundestag.de/btd/19/320/193203...
3,Entwurf eines ... Gesetzes zur Änderung des Bö...,https://dserver.bundestag.de/btd/19/318/193187...
4,Entwurf eines Gesetzes zur Änderung des Gesetz...,https://dserver.bundestag.de/btd/19/317/193171...


### Download the change laws

In [51]:
import os
from tqdm import tqdm

try:
    change_law_dir = "../data/change_laws/"
    os.mkdir(change_law_dir)
except FileExistsError:
    print("Directory already exists.")

Directory already exists.


In [54]:
for filepath in tqdm(change_law_df["fundstelle.pdf_url"].tolist()):
    response = requests.get(filepath)
    with open(f"{change_law_dir}{filepath.split('/')[-1]}", 'wb') as pdffile:
        pdffile.write(response.content)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [04:56<00:00,  1.19s/it]
